In [1]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import re, io, nltk, torch 
from nltk.corpus import stopwords
from numpy import linalg as LA
from numpy.linalg import norm
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
from sklearn.utils.class_weight import compute_class_weight
from gensim import utils, matutils  
from gensim.corpora.dictionary import Dictionary
from six import string_types, integer_types
from six.moves import zip, range
from numpy import linalg as LA
from sklearn.preprocessing import Normalizer
from scipy import stats
from gensim.utils import deprecated
from numpy import dot, float32 as REAL, memmap as np_memmap, \
    double, array, zeros, vstack, sqrt, newaxis, integer, \
    ndarray, sum as np_sum, prod, argmax
from collections import Counter
from scipy.spatial import distance
from numpy.random import binomial
from scipy.stats import bernoulli
import numpy as np

import keras
from keras.callbacks import Callback,ModelCheckpoint, ReduceLROnPlateau    
from keras import backend as K
from keras.preprocessing import sequence
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Embedding
from keras.layers import Conv1D, Conv2D, MaxPooling1D, Input
from keras.models import Model
from keras.layers import LSTM,GRU,Dense
from keras.utils import Sequence,to_categorical
from keras.preprocessing.sequence import pad_sequences

import transformers
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForMaskedLM,
    BertForSequenceClassification,
    BertTokenizer,
    CamembertConfig,
    CamembertForMaskedLM,
    CamembertTokenizer,
    DistilBertConfig,
    DistilBertForMaskedLM,
    DistilBertTokenizer,
    GPT2Config,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTConfig,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)

n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('There are',torch.cuda.device_count(),'GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt


def my_normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v/norm

#el peso max es igual a la clase mas grande
def det_samples(df, dict_class, c_weights=None):
    conteos= Counter(df)
    aumentos= dict()
    maxi= max(conteos.values())
    for clase in conteos:
        if conteos[clase]==maxi:
            class_maxi=clase
            
    if c_weights is None:        # uniform class weights
        for tipo in conteos:
            actual= conteos[tipo]
            if actual<maxi:
                aumentos[tipo]=int(maxi-actual)
            else:
                aumentos[tipo]=0
        return aumentos   
    else:
        try:
            weigths=my_normalize(c_weights)
            max_wei=max(weigths)
            class_max= list(weigths).index(max_wei)
            str_class=dict_class[class_max]
            samples_max=conteos[str_class]
            
            clase=0
            for peso in weigths:
                k=dict_class[clase]
                if peso == max_wei:
                    aumentos[k]= int(samples_max-conteos[k])
                else:
                    futuro=(samples_max/max_wei)*peso
                    aumentos[k]=int(futuro-conteos[k])
                clase+=1
            return aumentos
        except:
            print ("No se ha especificado el vector de pesos par balance de datos")
            
            
stoplist = stopwords.words("english")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def my_cleaner(post):      
    s= re.sub(r"http\S+", "html", post)      
    s= re.sub(r'#\w+ ?', "hashtag", s) 
    s= re.sub(r'@\w+ ?', "user", s)
    s= s.lower()    
    s=emoji_pattern.sub(r'', s)
    sl= list(s.split())  
    sl=[word for word in sl if word not in stoplist]
    s=' '.join([word for word in s.split() if word not in stoplist])
    return s, sl

def Sort_Tuple(tup):  
    return(sorted(tup, key = lambda x: x[1]))   


def predict_data(trained, x_train, x_val, x_test, etiq, etiq_v, etiq_t, name_model):
    etiq = etiq.astype("int")
    trainPredict = trained.predict(x_train, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq, trainPredict)  
    f1=f1_score(etiq, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    print ("")
    print ("Accuracy sobre Train", name_model, ":",acc)  
    print ("F1-score None sobre Train", name_model, ":",f1)
    print ("F1-score macro sobre Train", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t, matriz_t
    

Using TensorFlow backend.
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/casapanshop/anaconda2/envs/newpy3_tf1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [2]:
stoplist = stopwords.words("english")

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def my_cleaner(post):      
    s= re.sub(r"http\S+", "html", post)   ##########elimina http    
    s= re.sub(r'#\w+ ?', "hashtag", s) ########3
    s= re.sub(r'@\w+ ?', "user", s)##############    
    s= s.lower()    
    s=emoji_pattern.sub(r'', s)
    sl= list(s.split())  
    sl=[word for word in sl if word not in stoplist]
    s=' '.join([word for word in s.split() if word not in stoplist])
    return s, sl

#train =pd.read_csv("../Fine-Tuning/CSV_Stance/train_semeval_raw.csv")
val=pd.read_csv("../Fine-Tuning/CSV_Harassment/val_format.csv")
test=pd.read_csv("../Fine-Tuning/CSV_Harassment/test_format.csv")

print ("Conteo por harassment val\n", Counter(val['Label']))
print ("Conteo por harassment test\n", Counter(test['Label']))


Conteo por harassment val
 Counter({'NonH': 1493, 'SexualH': 525, 'IndirectH': 71, 'PhysicalH': 36})
Conteo por harassment test
 Counter({'NonH': 1601, 'SexualH': 340, 'IndirectH': 106, 'PhysicalH': 76})


In [3]:
MAX_LEN = 50

dir_path='../../model_save/Dos-Fases-all_Harassment/'
tokenizer_loaded = BertTokenizer.from_pretrained(dir_path)#'bert-base-uncased')
model_loaded = BertForSequenceClassification.from_pretrained(dir_path, num_labels=4)#'bert-base-uncased',  num_labels=4) 
idx_2_token = tokenizer_loaded.ids_to_tokens
archivo = open(dir_path+'vocab.txt','r')
Word2Index={word.strip():i for i,word in enumerate(archivo.readlines())}
M_BERT_space = model_loaded.bert.embeddings.word_embeddings.weight.detach().cpu().numpy()
transformer = Normalizer().fit(M_BERT_space) 
M_BERT_space=transformer.transform(M_BERT_space)

dict_cl=dict()
dict_cl[0]="NonH"
dict_cl[1]="IndirectH"
dict_cl[2]="PhysicalH"
dict_cl[3]="SexualH"

#nips
d_lab=dict()
d_lab["NonH"]=0
d_lab["IndirectH"]=1
d_lab["PhysicalH"]=2
d_lab["SexualH"]=3

etiq_v=[]
for et in val['Label'].values:
    etiq_v.append(d_lab[et])
n_labels_val = np.array(etiq_v)
y_val=to_categorical(n_labels_val,num_classes=4)
sentences_val = val['Tweet'].values
input_ids_val = []
for sent in sentences_val:
    encoded_sent = tokenizer_loaded.encode(my_cleaner(sent)[0],add_special_tokens = False)    
    input_ids_val.append(encoded_sent)
input_ids_val = pad_sequences(input_ids_val, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

etiq_t=[]
for et in test['Label'].values:
    etiq_t.append(d_lab[et])
n_labels_test = np.array(etiq_t)
y_test=to_categorical(n_labels_test,num_classes=4)
sentences_test = test['Tweet'].values
input_ids_test = []
for sent in sentences_test:
    encoded_sent = tokenizer_loaded.encode(my_cleaner(sent)[0],add_special_tokens = False)    
    input_ids_test.append(encoded_sent)
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], shape_val[1], 768))
i=0
for in_id in input_ids_val:
    x_val[i]=M_BERT_space[in_id]
    i+=1
    
shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], shape_test[1], 768))
i=0
for in_id in input_ids_test:
    x_test[i]=M_BERT_space[in_id]
    i+=1

x_test.shape

(2123, 50, 768)

In [4]:
x_test.shape

(2123, 50, 768)

In [5]:
def focal_loss(gamma=2., weights=1):   #weights np.asarray()
    weights= K.variable(weights)
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.clip(y_true, K.epsilon(),1)
        y_pred = K.clip(y_pred,K.epsilon(),1)
        return - K.sum(weights* K.pow(1. - y_pred, gamma)* y_true * K.log(y_pred), axis=-1) 
    return focal_loss_fixed

class Metrics(Callback):
    def on_train_begin(self, logs={}):
        self.val_f1s = []

    def on_epoch_end(self, batch, logs={}):
        prob = self.model.predict(self.validation_data[0])
        predict = np.squeeze(prob>=0.5)*1
        targ = np.squeeze(self.validation_data[1])
        f1s = f1_score(targ, predict, average='macro')
        self.val_f1s.append(f1s)
        #print(" - val_f1: %f " %(f1s))
        return

def plot_confusion_matrix(cm, target_names, title='Confusion matrix (f1-score)',cmap=None, normalize=True):
    
    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    
def predecir_modelos(bs,predichos_all):  #best fit
    temp=predichos_all.copy()
    final=[np.argmax(pred) for pred in temp]  
    confianzas=[temp[i][final[i]] for i in range(len(final))]
    predichos_new=[]
    for i in range(len(temp)):
        indice=final[i]%4
        predichos_new.append(int(indice))
    return predichos_new,final,confianzas    

def predecir_modelos_norm(bs,predichos_all):  #normalizado
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:   
        aux=np.ones(4)
        for i in range(1,6): #for machine pred
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux*np.asarray(sub)
            
        aux=aux/np.sum(aux)
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas
    
def predecir_modelos_average(bs,predichos_all): #average
    temp=predichos_all.copy()
    final=[]
    confianzas=[]
    for pred in temp:
        aux=np.zeros(4)
        for i in range(1,6):
            sub=pred[(4*(i-1)):(4*i)]
            aux=aux+np.asarray(sub)
            
        aux=aux/5.0 #dividido en el total de machines
        final.append(np.argmax(aux))
        confianzas.append(aux[np.argmax(aux)])  
           
    predichos_new=[]
    for i in range(len(temp)):
        predichos_new.append(final[i])
    return predichos_new,final,confianzas


def predict_data(trained, x_val, x_test, etiq_v, etiq_t, name_model):
    trainPredict = trained.predict(x_val, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc= accuracy_score(etiq_v, trainPredict)  
    f1=f1_score(etiq_v, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma=f1_score(etiq_v, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    #matriz=normalize(confusion_matrix(etiq_v, trainPredict))
    print ("")
    print ("Accuracy sobre Val", name_model, ":",acc)  
    print ("F1-score None sobre Val", name_model, ":",f1)
    print ("F1-score macro sobre Val", name_model, ":",f1_ma)
    ########################################
    trainPredict = trained.predict(x_test, batch_size=32)
    trainPredict=[np.argmax(pred) for pred in trainPredict]
    acc_t= accuracy_score(etiq_t, trainPredict)  
    f1_t=f1_score(etiq_t, trainPredict, average=None)  # labels=np.unique(trainPredict)
    f1_ma_t=f1_score(etiq_t, trainPredict, average='macro')  # labels=np.unique(trainPredict)
    matriz_t=normalize(confusion_matrix(etiq_t, trainPredict))
    print ("")
    print ("Accuracy sobre Test", name_model, ":",acc_t)  
    print ("F1-score None sobre Test", name_model, ":",f1_t)
    print ("F1-score macro sobre Test", name_model, ":",f1_ma_t)
    
    return f1_ma_t, f1_t, acc_t #, matriz_t

In [6]:
x_test.shape

(2123, 50, 768)

# Baselines masking=0.0 Sin class weights

In [14]:
prob=0.0
path='../Operador Data Augmentation/exec_baselines_harass/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'_wo/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'_wo/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'_wo/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'_wo/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'_wo/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8847058823529412
F1-macro: 0.45163846280059516
F1-score SDQC: 0.45163846280059516

Committee Norm
Accuracy: 0.8927058823529411
F1-macro: 0.4750348869663363
F1-score SDQC: 0.4750348869663363

Committee Voting
Accuracy: 0.8931764705882353
F1-macro: 0.47511232555063826
F1-score SDQC: 0.47511232555063826
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8398492699010833
F1-macro: 0.41302354448906176
F1-score SDQC: 0.41302354448906176

Committee Norm
Accuracy: 0.8450306170513424
F1-macro: 0.4108621067231185
F1-score SDQC: 0.4108621067231185

Committee Voting
Accuracy: 0.8469147432878003
F1-macro: 0.4130111197088888
F1-score SDQC: 0.4130111197088888

---------------- EJECUCIÓN #2----------------
Agregando predicci

In [15]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.8727529411764706
[VAL] F1-score SDQC Best comité: [0.92465594 0.03186898 0.04055484 0.81523999]
[VAL] F1 macro Best comité: 0.4530799387098111
-----------
[VAL] Acc Norm comité: 0.8796235294117647
[VAL] F1-score SDQC Norm comité: [0.92688146 0.01614612 0.08620388 0.82857504]
[VAL] F1 macro Norm comité: 0.46445162452718836
-----------
[VAL] Acc Voting comité: 0.8835764705882352
[VAL] F1-score SDQC Voting comité: [0.92966982 0.02133333 0.06709576 0.83855841]
[VAL] F1 macro Voting comité: 0.4641643295550222

[TEST] Acc Best comité: 0.8335374470089496
[TEST] F1-score SDQC Best comité: [0.90564302 0.01072125 0.03501866 0.6737307 ]
[TEST] F1 macro Best comité: 0.4062784056503955
-----------
[TEST] Acc Norm comité: 0.8375883184173339
[TEST] F1-score SDQC Norm comité: [0.90660699 0.00363636 0.01948052 0.68426389]
[TEST] F1 macro Norm comité: 0.4034969400799432
-----------
[TEST] Acc Voting comité: 0.8404145077720206
[TEST] F1-score SDQC Voting comité: [0.90827782 0.007

In [17]:
np.sum(fs_macro_test['cnn2'])/5

0.3946579417939224

### No data augmentation with class weights

In [18]:
prob=0.0
path='../Operador Data Augmentation/exec_baselines_harass/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.856
F1-macro: 0.4674628152431399
F1-score SDQC: 0.4674628152431399

Committee Norm
Accuracy: 0.8861176470588236
F1-macro: 0.4897821832896734
F1-score SDQC: 0.4897821832896734

Committee Voting
Accuracy: 0.8856470588235295
F1-macro: 0.4836263144462799
F1-score SDQC: 0.4836263144462799
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8257183231276496
F1-macro: 0.4615048818711459
F1-score SDQC: 0.4615048818711459

Committee Norm
Accuracy: 0.8455016486104568
F1-macro: 0.45242057133090174
F1-score SDQC: 0.45242057133090174

Committee Voting
Accuracy: 0.8520960904380593
F1-macro: 0.47603460336458225
F1-score SDQC: 0.47603460336458225

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy:

In [19]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.8656941176470588
[VAL] F1-score SDQC Best comité: [0.92554721 0.07382042 0.06402727 0.81396651]
[VAL] F1 macro Best comité: 0.46934035213914044
-----------
[VAL] Acc Norm comité: 0.8828235294117648
[VAL] F1-score SDQC Norm comité: [0.93129419 0.07021738 0.05912418 0.84055507]
[VAL] F1 macro Norm comité: 0.4752977020925414
-----------
[VAL] Acc Voting comité: 0.8815058823529414
[VAL] F1-score SDQC Voting comité: [0.93163735 0.06928308 0.04695681 0.83968365]
[VAL] F1 macro Voting comité: 0.47189022469567743

[TEST] Acc Best comité: 0.8290155440414508
[TEST] F1-score SDQC Best comité: [0.9065706  0.08448481 0.06193971 0.69192439]
[TEST] F1 macro Best comité: 0.436229876957572
-----------
[TEST] Acc Norm comité: 0.8417333961375412
[TEST] F1-score SDQC Norm comité: [0.91018925 0.04464833 0.03240069 0.72033679]
[TEST] F1 macro Norm comité: 0.4268937658075921
-----------
[TEST] Acc Voting comité: 0.8439001413094677
[TEST] F1-score SDQC Voting comité: [0.91146773 0.070

In [20]:
np.sum(fs_macro_test['cnn2'])/5

0.36815395067255396

### Augmentation 0.15

In [21]:
prob=0.15
path='../Operador Data Augmentation/exec_baselines_harass/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)



---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8696470588235294
F1-macro: 0.4790917114519783
F1-score SDQC: 0.4790917114519783

Committee Norm
Accuracy: 0.8856470588235295
F1-macro: 0.4573743908834216
F1-score SDQC: 0.4573743908834216

Committee Voting
Accuracy: 0.8912941176470588
F1-macro: 0.4675869805712206
F1-score SDQC: 0.4675869805712206
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8186528497409327
F1-macro: 0.44014810212871286
F1-score SDQC: 0.44014810212871286

Committee Norm
Accuracy: 0.8426754592557701
F1-macro: 0.419759828271551
F1-score SDQC: 0.419759828271551

Committee Voting
Accuracy: 0.8473857748469148
F1-macro: 0.42889216613261777
F1-score SDQC: 0.42889216613261777

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fi

In [22]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.815435294117647
[VAL] F1-score SDQC Best comité: [0.8816178  0.04871317 0.11390293 0.78105582]
[VAL] F1 macro Best comité: 0.4563224292773483
-----------
[VAL] Acc Norm comité: 0.8769882352941176
[VAL] F1-score SDQC Norm comité: [0.92541438 0.02099515 0.05005102 0.83055087]
[VAL] F1 macro Norm comité: 0.4567528538419047
-----------
[VAL] Acc Voting comité: 0.8856470588235295
[VAL] F1-score SDQC Voting comité: [0.93163373 0.02106519 0.04859474 0.84832898]
[VAL] F1 macro Voting comité: 0.4624056588287239

[TEST] Acc Best comité: 0.7708902496467264
[TEST] F1-score SDQC Best comité: [0.86358491 0.07892092 0.06957839 0.61845676]
[TEST] F1 macro Best comité: 0.407635245463999
-----------
[TEST] Acc Norm comité: 0.8309938765897316
[TEST] F1-score SDQC Norm comité: [0.90303666 0.03559777 0.02039117 0.67873738]
[TEST] F1 macro Norm comité: 0.4094407450035501
-----------
[TEST] Acc Voting comité: 0.8394724446537918
[TEST] F1-score SDQC Voting comité: [0.90824274 0.049278

In [23]:
np.sum(fs_macro_test['cnn2'])/5

0.3975311115881673

### Augmentation 0.5

In [24]:
prob=0.5
path='../Operador Data Augmentation/exec_baselines_harass/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)


---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8202352941176471
F1-macro: 0.45051219389033403
F1-score SDQC: 0.45051219389033403

Committee Norm
Accuracy: 0.8611764705882353
F1-macro: 0.4496277138087339
F1-score SDQC: 0.4496277138087339

Committee Voting
Accuracy: 0.8729411764705882
F1-macro: 0.4667060088822116
F1-score SDQC: 0.4667060088822116
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.7508243052284503
F1-macro: 0.38369145555527484
F1-score SDQC: 0.38369145555527484

Committee Norm
Accuracy: 0.7974564295807819
F1-macro: 0.3997421755382652
F1-score SDQC: 0.3997421755382652

Committee Voting
Accuracy: 0.8162976919453604
F1-macro: 0.4208996298713136
F1-score SDQC: 0.4208996298713136

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best 

In [25]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.8476235294117647
[VAL] F1-score SDQC Best comité: [0.90622688 0.03528168 0.09238461 0.78992867]
[VAL] F1 macro Best comité: 0.45595545959756456
-----------
[VAL] Acc Norm comité: 0.8711529411764705
[VAL] F1-score SDQC Norm comité: [0.92170342 0.02696286 0.06714205 0.81722844]
[VAL] F1 macro Norm comité: 0.458259189546279
-----------
[VAL] Acc Voting comité: 0.8815058823529413
[VAL] F1-score SDQC Voting comité: [0.92933741 0.04213997 0.0661053  0.83731628]
[VAL] F1 macro Voting comité: 0.46872474177349066

[TEST] Acc Best comité: 0.7953838907206783
[TEST] F1-score SDQC Best comité: [0.88310553 0.05065818 0.04864069 0.60916026]
[TEST] F1 macro Best comité: 0.39789116617633685
-----------
[TEST] Acc Norm comité: 0.8189354686764013
[TEST] F1-score SDQC Norm comité: [0.89785981 0.04231314 0.01904762 0.64311181]
[TEST] F1 macro Norm comité: 0.40058309546844806
-----------
[TEST] Acc Voting comité: 0.8286387187941593
[TEST] F1-score SDQC Voting comité: [0.90355939 0.0

In [26]:
np.sum(fs_macro_test['cnn2'])/5

0.40812622341881843

### Augmentation 0.85

In [27]:
prob=0.85
path='../Operador Data Augmentation/exec_baselines_harass/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/0.15/rnn1_'+str(i+1)+'-exec_w4_app1_Top_'+str(prob)+'_'+str(TOPN)+'.h5'
#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+str(prob)+'/cnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+str(prob)+'/cnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+str(prob)+'/rnn1_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+str(prob)+'/rnn2_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+str(prob)+'/rnn3_'+str(j+1)+'-exec_w4_app1_Top_'+str(prob)+'_1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)


---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8672941176470588
F1-macro: 0.4443142865487368
F1-score SDQC: 0.4443142865487368

Committee Norm
Accuracy: 0.8818823529411765
F1-macro: 0.45330817176540916
F1-score SDQC: 0.45330817176540916

Committee Voting
Accuracy: 0.8804705882352941
F1-macro: 0.45188218221567067
F1-score SDQC: 0.45188218221567067
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.802637776731041
F1-macro: 0.39937751253523657
F1-score SDQC: 0.39937751253523657

Committee Norm
Accuracy: 0.8214790390956194
F1-macro: 0.40853028010480863
F1-score SDQC: 0.40853028010480863

Committee Voting
Accuracy: 0.8313707018370231
F1-macro: 0.4003174588906891
F1-score SDQC: 0.4003174588906891

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Be

In [28]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.8407529411764706
[VAL] F1-score SDQC Best comité: [0.9015838  0.0447579  0.07533924 0.78904568]
[VAL] F1 macro Best comité: 0.45268165655798875
-----------
[VAL] Acc Norm comité: 0.8714352941176471
[VAL] F1-score SDQC Norm comité: [0.92238185 0.01067426 0.04823197 0.82162536]
[VAL] F1 macro Norm comité: 0.4507283586586342
-----------
[VAL] Acc Voting comité: 0.8768
[VAL] F1-score SDQC Voting comité: [0.9267127  0.02033333 0.04742924 0.83010839]
[VAL] F1 macro Voting comité: 0.4561459166487801

[TEST] Acc Best comité: 0.7745642958078192
[TEST] F1-score SDQC Best comité: [0.86851517 0.08654561 0.03896219 0.59874164]
[TEST] F1 macro Best comité: 0.3981911532112018
-----------
[TEST] Acc Norm comité: 0.8116815826660386
[TEST] F1-score SDQC Norm comité: [0.89237518 0.06032783 0.0101915  0.64683535]
[TEST] F1 macro Norm comité: 0.4024324631170127
-----------
[TEST] Acc Voting comité: 0.8211022138483278
[TEST] F1-score SDQC Voting comité: [0.89812982 0.05373344 0.    

In [29]:
np.sum(fs_macro_test['cnn2'])/5

0.40397515043187593

### SMOTE

In [7]:
#SMOTE
shape_val=np.asarray(input_ids_val).shape
x_val=np.zeros((shape_val[0], 768))
i=0
for in_id in input_ids_val:
    vector=np.zeros(768)
    n = np.sum(in_id != 0)
    for idv in in_id:
        if idv!=0:
            vector+=M_BERT_space[idv]
    x_val[i]=(vector/n)
    i+=1  

shape_test=np.asarray(input_ids_test).shape
x_test=np.zeros((shape_test[0], 768))
i=0
for in_id in input_ids_test:
    vector=np.zeros(768)
    n = np.sum(in_id != 0)
    for idv in in_id:
        if idv!=0:
            vector+=M_BERT_space[idv]
    x_test[i]=(vector/n)
    i+=1 
    
from imblearn.over_sampling import SMOTE

x_val=np.expand_dims(x_val, axis=-1)
x_test=np.expand_dims(x_test, axis=-1)

In [9]:
path='../Operador Data Augmentation/exec_baselines_harass/'
bs=32
    
fs_macro_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_val={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

fs_macro_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
fs_none_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
accs_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}
confusions_test={'cnn1':[], 'cnn2':[], 'rnn1':[],'rnn2':[],'rnn3':[]}

best={'acc':[], 'none':[], 'macro':[]}
norm={'acc':[], 'none':[], 'macro':[]}
voting={'acc':[], 'none':[], 'macro':[]}
best_test={'acc':[], 'none':[], 'macro':[]}
norm_test={'acc':[], 'none':[], 'macro':[]}
voting_test={'acc':[], 'none':[], 'macro':[]}

#exec_baselines/SMOTE/cnn2_'+str(i+1)+'-exec_w4_app1.h5
#cnn1_4239-exec_w4_app1.h5
for j in range(5):
    print ("")
    print ("---------------- EJECUCIÓN #"+str(j+1)+'----------------')
    cnn1= load_model(path+'SMOTE/cnn1_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()})  
    cnn2= load_model(path+'SMOTE/cnn2_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn1= load_model(path+'SMOTE/rnn1_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn2= load_model(path+'SMOTE/rnn2_'+str(j+1)+'-exec_w4_app1.h5', custom_objects={'FocalLoss': focal_loss, 'focal_loss_fixed': focal_loss()}) 
    rnn3= load_model(path+'SMOTE/rnn3_'+str(j+1)+'-exec_w4_app1.h5') 
    
    list_models=['cnn1', 'cnn2', 'rnn1', 'rnn2', 'rnn3']
    index_models=np.arange(5)
    dict_models=dict((key, value) for (key, value) in zip(index_models,list_models))
    modelos=[cnn1, cnn2, rnn1, rnn2, rnn3]
    ind=np.arange(5)
    dict_trainedModel=dict((key, value) for (key, value) in zip(ind,modelos))
    
    print ("Agregando predicciones Val set")
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_val, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_v, predicho)  
        f1_ma=f1_score(etiq_v, predicho, average='macro')
        f1_no=f1_score(etiq_v, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_v, predicho))
        fs_macro_val[dict_models[i]].append(f1_ma)
        fs_none_val[dict_models[i]].append(f1_no)
        accs_val[dict_models[i]].append(acc)
        confusions_val[dict_models[i]].append(matriz)
        
    predicciones_all_val=[]
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_val.append(dict_trainedModel[i].predict(x_val, batch_size=bs))
    predichos_all_val=np.concatenate(np.asarray(predicciones_all_val),axis=-1)
    
    print ("Agregando predicciones Test set", dict_models[i])    
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicho=dict_trainedModel[i].predict(x_test, batch_size=bs)
        predicho=[np.argmax(pred) for pred in predicho]
        acc= accuracy_score(etiq_t, predicho)  
        f1_ma=f1_score(etiq_t, predicho, average='macro')
        f1_no=f1_score(etiq_t, predicho, average=None)
        matriz=normalize(confusion_matrix(etiq_t, predicho))
        fs_macro_test[dict_models[i]].append(f1_ma)
        fs_none_test[dict_models[i]].append(f1_no)
        accs_test[dict_models[i]].append(acc)
        confusions_test[dict_models[i]].append(matriz)

    predicciones_all_test=[]
    bs=32
    for i in dict_trainedModel.keys(): 
        a_evaluar=dict_models[i]
        predicciones_all_test.append(dict_trainedModel[i].predict(x_test, batch_size=bs))
    predichos_all_test=np.concatenate(np.asarray(predicciones_all_test),axis=-1)
    
    print ("--------VALIDATION SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best['acc'].append(acc_comite)
    best['macro'].append(f1)
    best['none'].append(f1_no)
    
    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm['acc'].append(acc_comite)
    norm['macro'].append(f1)
    norm['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_val)
    acc_comite= accuracy_score(etiq_v, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_v, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_v, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting['acc'].append(acc_comite)
    voting['macro'].append(f1)
    voting['none'].append(f1_no)
    
    print ("--------TESTING SET--------")
    
    print ("\nCommittee Best Fit")
    trainPredicho, maxPredicho, confianzas = predecir_modelos(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    best_test['acc'].append(acc_comite)
    best_test['macro'].append(f1)
    best_test['none'].append(f1_no)

    print ("\nCommittee Norm")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_norm(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None) 
    print("F1-score SDQC:",f1)
    norm_test['acc'].append(acc_comite)
    norm_test['macro'].append(f1)
    norm_test['none'].append(f1_no)

    print ("\nCommittee Voting")
    trainPredicho, maxPredicho, confianzas = predecir_modelos_average(bs,predichos_all_test)
    acc_comite= accuracy_score(etiq_t, trainPredicho)  
    print("Accuracy:",acc_comite)
    f1=f1_score(etiq_t, trainPredicho, average='macro') 
    print("F1-macro:",f1)
    f1_no=f1_score(etiq_t, trainPredicho, average=None)  
    print("F1-score SDQC:",f1)
    voting_test['acc'].append(acc_comite)
    voting_test['macro'].append(f1)
    voting_test['none'].append(f1_no)


---------------- EJECUCIÓN #1----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fit
Accuracy: 0.8701176470588236
F1-macro: 0.47879294914588266
F1-score SDQC: 0.47879294914588266

Committee Norm
Accuracy: 0.8790588235294118
F1-macro: 0.4638909962624448
F1-score SDQC: 0.4638909962624448

Committee Voting
Accuracy: 0.8790588235294118
F1-macro: 0.4634854926222458
F1-score SDQC: 0.4634854926222458
--------TESTING SET--------

Committee Best Fit
Accuracy: 0.8271314178049929
F1-macro: 0.3942449103499841
F1-score SDQC: 0.3942449103499841

Committee Norm
Accuracy: 0.8290155440414507
F1-macro: 0.38745754520481335
F1-score SDQC: 0.38745754520481335

Committee Voting
Accuracy: 0.8308996702779087
F1-macro: 0.390646406775439
F1-score SDQC: 0.390646406775439

---------------- EJECUCIÓN #2----------------
Agregando predicciones Val set
Agregando predicciones Test set rnn3
--------VALIDATION SET--------

Committee Best Fi

In [10]:
print ("[VAL] Acc Best comité:",np.sum(np.asarray(best['acc'])/5.0))
temp=np.zeros(4)
for result in best['none']:
    temp+=result
print ("[VAL] F1-score SDQC Best comité:" ,temp/5)
print ("[VAL] F1 macro Best comité:",np.sum(np.asarray(best['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Norm comité:",np.sum(np.asarray(norm['acc'])/5.0))
temp=np.zeros(4)
for result in norm['none']:
    temp+=result
print ("[VAL] F1-score SDQC Norm comité:" ,temp/5)
print ("[VAL] F1 macro Norm comité:",np.sum(np.asarray(norm['macro'])/5.0))
print ("-----------")
print ("[VAL] Acc Voting comité:",np.sum(np.asarray(voting['acc'])/5.0))
temp=np.zeros(4)
for result in voting['none']:
    temp+=result
print ("[VAL] F1-score SDQC Voting comité:" ,temp/5)
print ("[VAL] F1 macro Voting comité:",np.sum(np.asarray(voting['macro'])/5.0))

print ("")

print ("[TEST] Acc Best comité:",np.sum(np.asarray(best_test['acc'])/5.0))
temp=np.zeros(4)
for result in best_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Best comité:" ,temp/5)
print ("[TEST] F1 macro Best comité:",np.sum(np.asarray(best_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Norm comité:",np.sum(np.asarray(norm_test['acc'])/5.0))
temp=np.zeros(4)
for result in norm_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Norm comité:" ,temp/5)
print ("[TEST] F1 macro Norm comité:",np.sum(np.asarray(norm_test['macro'])/5.0))
print ("-----------")
print ("[TEST] Acc Voting comité:",np.sum(np.asarray(voting_test['acc'])/5.0))
temp=np.zeros(4)
for result in voting_test['none']:
    temp+=result
print ("[TEST] F1-score SDQC Voting comité:" ,temp/5)
print ("[TEST] F1 macro Voting comité:",np.sum(np.asarray(voting_test['macro'])/5.0))


[VAL] Acc Best comité: 0.8676705882352941
[VAL] F1-score SDQC Best comité: [0.92101027 0.03874441 0.08686944 0.813984  ]
[VAL] F1 macro Best comité: 0.46515202740427397
-----------
[VAL] Acc Norm comité: 0.8704941176470589
[VAL] F1-score SDQC Norm comité: [0.92022483 0.00540541 0.06959895 0.81213786]
[VAL] F1 macro Norm comité: 0.45184176349419913
-----------
[VAL] Acc Voting comité: 0.8738823529411766
[VAL] F1-score SDQC Voting comité: [0.92255912 0.00540541 0.07831591 0.82088399]
[VAL] F1 macro Voting comité: 0.4567911073164252

[TEST] Acc Best comité: 0.8213848327837965
[TEST] F1-score SDQC Best comité: [0.89955769 0.02368094 0.00493827 0.62872001]
[TEST] F1 macro Best comité: 0.3892242271320441
-----------
[TEST] Acc Norm comité: 0.820819594912859
[TEST] F1-score SDQC Norm comité: [0.89718446 0.         0.         0.60125427]
[TEST] F1 macro Norm comité: 0.3746096830409472
-----------
[TEST] Acc Voting comité: 0.8256241168158267
[TEST] F1-score SDQC Voting comité: [0.90013035 0.003

In [11]:
np.sum(fs_macro_test['cnn2'])/5

0.3972362262138721